<a href="https://colab.research.google.com/github/solmak97/Sudoku_Detection/blob/master/yolov3_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#YOLOv3 Customized Object Detection with Darknet Framework

This notebook shows how to train YOLOv3 for custom object detection on google colab utilizing the free GPU resources.A very well documented tutorial on how to train YOLOv3 to detect custom objects can be founded on Github from  [AlexeyAB](https://github.com/AlexeyAB/darknet.git) and [Youtube](https://www.youtube.com/watch?v=hTCmL3S4Obw&t=976s).
To know the hardware specification provided in GOOGLE COLAB environment run this command ([Source](https://colab.research.google.com/drive/151805XTDg--dgHb3-AXJCpnWaqRhop_2)):

## Step 1 : Dataset Preparation

### Annotation
To train a detector with YOLOv3 dataset to be included images associated with a label (a txt file). Labels file defines the object class and coordinates of object bounding box in an image.

To create label files from images, [LableImg](https://github.com/tzutalin/labelImg) is used. Annotations are saved as .txt files in YOLO format, the format used by YOLOv3 .A txt file of YOLO format to be saved in the same folder as your image is with same name. A file named "classes.txt"automaticallly saved to that folder. "classes.txt" defines the list of class names that labels refer to.



##Step 2: Enabling GPU within your notebook
i) Click Edit at top left of your notebook

ii) Click Notebook Settings within dropdown

iii) Under 'Hardware Accelerator' select GPU and then hit Save

To know the hardware specification provided in GOOGLE COLAB environment run this command:

In [ ]:
!nvidia-smi

##Step 3: Upload files 

Files to be uploaded on your google drive before. Then use following command:

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

##Step 4: Split Dataset train/validation
To avoid over-fitting and achieved an objective evaluation regarding our model, we need to split our dataset into training set and validation set. We will use a “train.txt” file and “test.txt” to define the directory path of our training and validation images.[See](https://github.com/jakkcoder/training_yolo_custom_object_detection_files)



In [ ]:
! python custom_data/creating-files-data-and-name.py

##Step 5:Creating files labelled_data.data and classes.names
In the next step, we need to create file “classes.names” in the directory, with 
object names as each new line. For instance, in our case we need to detect Sudoku only so the first line should be “sudoku”.Finally, create file “labelled_data.data”.[See](https://github.com/jakkcoder/training_yolo_custom_object_detection_files)

In [ ]:
! python custom_data/creating-train-and-test-txt-files.py

##Step 6: Cloning and Building Darknet

The following cells will clone darknet from AlexeyAB's famous repository, adjust the Makefile to enable OPENCV , GPU and CUDNN for darknet and then build darknet.

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
#change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

## Step 7 :Customized YOLOv3 Configuration
Here the approach to train a customized object detection model is Transfer Learning. We use a pre-trained model from darknet and try to modfiy last layers of the model to customized it for our dataset.To see the internal layers of YOLOv3 you can use the [Nectron]( https://netron.app/) application.
Turning to yolov3.cf file from [AlexeyAB](https://github.com/AlexeyAB/darknet/blob/master/cfg/yolov3.cfg), follwoing items to be modified for our custom training model.Create a copy of “yolov3.cfg” in the same folder and rename it.Edit the file as below instruction:

Line 8 & 9: 
width = 416, height = 416 (Network Size)

Line 20 
max_batches = 6000 (Recommended by AlexeyAB )

Line 22 
steps = 4800 , 5400 (Recommended by AlexeyAB )

Line 603, 689, 776: 
filters = 18    **(Calculate the number of filters in the last layers of YOLOv3 with this formula: No of filters = (classes + 5)*3 )**
Line 610, 696, 783: 
classes = 1 ("Sudoku")

Line 609 ,695,782:

anchors = 127, 125, 163, 182,184,204,251,240,253,286,280, 288,327,365, 368, 309,372, 384

*Size of anchor boxes to be calculate for your custom dataset :








In [ ]:
!./darknet detector calc_anchors custom_data/labelled_data.data -num_of_clusters 9 -width 416 -height 416


In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom_train.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_custom_train.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_custom_train.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_custom_train.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_custom_train.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_custom_train.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_custom_train.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_custom_train.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_custom_train.cfg
!sed -i '609 s@anchors=10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326 @anchors=127, 125, 163, 182,184,204,251,240,253,286,280, 288,327,365, 368, 309,372, 384@' cfg/yolov3_custom_train.cfg
!sed -i '695 s@anchors=10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326 @anchors=127, 125, 163, 182,184,204,251,240,253,286,280, 288,327,365, 368, 309,372, 384@' cfg/yolov3_custom_train.cfg
!sed -i '782 s@anchors=10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326 @anchors=127, 125, 163, 182,184,204,251,240,253,286,280, 288,327,365, 368, 309,372, 384@' cfg/yolov3_custom_train.cfg

##Step 8: Download pre-trained YOLOv3 weights

Download darknet pre-trained model [darknet53.conv.74](https://pjreddie.com/media/files/darknet53.conv.74)

##Step 9: Train and Test the model
Then, change directory to the darknet folder and initilize the darknet command which is built in step 2.

In [ ]:
%cd '/content/drive/MyDrive/yolo_custom_model_training/darknet'
!chmod +x ./darknet 

###Train

The train command starts trainging from initial weights of darknet53, and it saves the weights in every 100 iteration until 1000th iteration. Then, it saves weights every 1000 iterations : 

In [ ]:
!./darknet detector train custom_data/labelled_data.data  cfg/yolov3_custom_train.cfg backup/darknet53.conv.74 -dont_show


To underestand which model is better, mAP values for each model to be checked:

In [ ]:
!./darknet detector map custom_data/labelled_data.data  cfg/yolov3_custom_train.cfg backup/yolov3_custom_1000.weights


Evaluate Model:

Loss= 0.02

Mean Average Precision =97.44%

Average IoU=82.7%


###Test

For test the model, create a copy of customized train .cfg file (from step 7) and uncomment lines 2,3 and 4 . Set batch and subdividons to one. Comment lines 5 , 6 and 7. Then run the folowing command using latest model with the best mAP :

In [ ]:
!./darknet detector test custom_data/labelled_data.data  cfg/yolov3_custom_test.cfg backup/yolov3_custom_last.weights data/img987.jpg -thresh 0.1
